# Book Recommender System using Popularity & Collaborative Filtering

## Required Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load Data

In [2]:
books = pd.read_csv('data/Books.csv')
users = pd.read_csv('data/Users.csv')
ratings = pd.read_csv('data/Ratings.csv')

C:\Users\yash.vora\AppData\Local\Temp\ipykernel_20932\927319382.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('data/Books.csv')


In [3]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [5]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


## Basic Info of Data

In [6]:
print(books.info())
print()
print(users.info())
print()
print(ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), obj

## Check Null Values

In [7]:
print(books.isnull().sum())
print()
print(users.isnull().sum())
print()
print(ratings.isnull().sum())

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

User-ID          0
Location         0
Age         110762
dtype: int64

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64


In [8]:
len(books), len(users), len(ratings)

(271360, 278858, 1149780)

## Popularity Based Recommendation System
Top 100 books with,
- Highest Average Ratings
- Select book with minimum 250 ratings

In [9]:
ratings_with_book_name = books.merge(ratings, on='ISBN')
ratings_with_book_name.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0


In [10]:
# Count of ratings on each book
num_ratings = ratings_with_book_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_ratings.rename(columns={'Book-Rating': 'Num-Ratings'}, inplace=True)
num_ratings

,Book-Title,Num-Ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [11]:
# Average ratings of each book
avg_ratings = ratings_with_book_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_ratings.rename(columns={'Book-Rating': 'Avg-Ratings'}, inplace=True)
avg_ratings

,Book-Title,Avg-Ratings
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [12]:
popular_df = num_ratings.merge(avg_ratings, on='Book-Title')
popular_df

,Book-Title,Num-Ratings,Avg-Ratings
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [13]:
popular_df = popular_df[popular_df['Num-Ratings'] >= 250].sort_values('Avg-Ratings', ascending=False)[:100].reset_index(drop=True)
popular_df

,Book-Title,Num-Ratings,Avg-Ratings
0,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
1,Harry Potter and the Goblet of Fire (Book 4),387,5.824289
2,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410
3,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441
4,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453
...,...,...,...
95,The Beach House,435,3.347126
96,The Hours: A Novel,294,3.346939
97,Icy Sparks,309,3.346278
98,Angela's Ashes (MMP) : A Memoir,326,3.337423


In [17]:
popular_df = popular_df.merge(books, on='Book-Title').drop_duplicates('Book-Title')[['Book-Title', 'Book-Author', 'Image-URL-M', 'Num-Ratings', 'Avg-Ratings']].reset_index(drop=True)
popular_df

,Book-Title,Book-Author,Image-URL-M,Num-Ratings,Avg-Ratings
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
1,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
2,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
3,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
4,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453
...,...,...,...,...,...
95,The Beach House,James Patterson,http://images.amazon.com/images/P/0446612545.0...,435,3.347126
96,The Hours: A Novel,Michael Cunningham,http://images.amazon.com/images/P/0312305060.0...,294,3.346939
97,Icy Sparks,Gwyn Hyman Rubio,http://images.amazon.com/images/P/0142000205.0...,309,3.346278
98,Angela's Ashes (MMP) : A Memoir,Frank McCourt,http://images.amazon.com/images/P/0684872153.0...,326,3.337423


In [19]:
import pickle

with open('popular_books.pkl', 'wb') as f:
    pickle.dump(popular_df, f)

## Collaborative Filtering Based Recommendation System
We have to create pivot table where index = book names, columns = users, values = ratings.<br>
But there are some conditions:<br>
1 Select user who have given more then 200 ratings.<br>
2 Select books that have at least 50 ratings.

In [67]:
#  Counting ratings given by each user
user_ratings = ratings_with_book_name.groupby('User-ID').count()['Book-Rating'].reset_index()
user_ratings.head()

,User-ID,Book-Rating
0,2,1
1,8,17
2,9,3
3,10,1
4,12,1


In [68]:
# Users who have given more then 200 ratings
user_ratings = user_ratings[user_ratings['Book-Rating'] > 200]
user_ratings.head()

,User-ID,Book-Rating
87,254,300
698,2276,456
862,2766,269
919,2977,227
1033,3363,890


In [75]:
user_ratings.shape

(811, 2)

In [74]:
filtered_ratings = ratings_with_book_name[ratings_with_book_name['User-ID'].isin(user_ratings['User-ID'])]
filtered_ratings

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
6,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,85526,0
7,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,96054,0
10,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,177458,0
21,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,110912,10
...,...,...,...,...,...,...,...,...,...,...
1031124,0231128444,Slow Food(The Case For Taste),Carlo Petrini,2003,Columbia University Press,http://images.amazon.com/images/P/0231128444.0...,http://images.amazon.com/images/P/0231128444.0...,http://images.amazon.com/images/P/0231128444.0...,275970,0
1031125,0520242335,Strong Democracy : Participatory Politics for ...,Benjamin R. Barber,2004,University of California Press,http://images.amazon.com/images/P/0520242335.0...,http://images.amazon.com/images/P/0520242335.0...,http://images.amazon.com/images/P/0520242335.0...,275970,0
1031126,0762412119,"Burpee Gardening Cyclopedia: A Concise, Up to ...",Allan Armitage,2002,Running Press Book Publishers,http://images.amazon.com/images/P/0762412119.0...,http://images.amazon.com/images/P/0762412119.0...,http://images.amazon.com/images/P/0762412119.0...,275970,0
1031127,1582380805,Tropical Rainforests: 230 Species in Full Colo...,"Allen M., Ph.D. Young",2001,Golden Guides from St. Martin's Press,http://images.amazon.com/images/P/1582380805.0...,http://images.amazon.com/images/P/1582380805.0...,http://images.amazon.com/images/P/1582380805.0...,275970,0


In [80]:
# Counting ratings on each book
book_ratings = filtered_ratings.groupby('Book-Title').count()['Book-Rating'].reset_index()
book_ratings.head()

,Book-Title,Book-Rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [82]:
# Books on which there are more then 50 ratings
book_ratings = book_ratings[book_ratings['Book-Rating'] >= 50]
book_ratings.head()

,Book-Title,Book-Rating
484,1984,70
518,1st to Die: A Novel,160
653,2nd Chance,122
796,4 Blondes,70
1116,A Bend in the Road,114


In [83]:
book_ratings.shape

(706, 2)

In [160]:
# This are books where each user has given more then 200 ratings and each book have more then 50 ratings
final_df = filtered_ratings[filtered_ratings['Book-Title'].isin(book_ratings['Book-Title'])].reset_index(drop=True)
final_df = final_df[['Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]
final_df

,Book-Title,Book-Author,User-ID,Book-Rating
0,The Kitchen God's Wife,Amy Tan,11676,9
1,The Kitchen God's Wife,Amy Tan,36836,0
2,The Kitchen God's Wife,Amy Tan,46398,9
3,The Kitchen God's Wife,Amy Tan,113270,0
4,The Kitchen God's Wife,Amy Tan,113519,0
...,...,...,...,...
58581,Echoes,Nancy Morse,238781,0
58582,I Know Why the Caged Bird Sings,Maya Angelou,239594,8
58583,The Promise,CHAIM POTOK,239594,7
58584,The Pillars of the Earth,Ken Follett,240144,0


In [93]:
# For collaborative filtering we have to create pivot table
# index = Book-Title, columns = User-ID, values = Book-Rating
pt = pd.pivot_table(data=final_df, index='Book-Title', columns='User-ID', values='Book-Rating')
pt.fillna(0, inplace=True)
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Cosine Similarity - To find similarity of each book with other books

In [94]:
from sklearn.metrics.pairwise import cosine_similarity

In [96]:
# Similarity score of each book with other books
similarity_score = cosine_similarity(pt)
similarity_score

array([[1.        , 0.10255025, 0.01220856, ..., 0.12110367, 0.07347567,
        0.04316046],
       [0.10255025, 1.        , 0.2364573 , ..., 0.07446129, 0.16773875,
        0.14263397],
       [0.01220856, 0.2364573 , 1.        , ..., 0.04558758, 0.04938579,
        0.10796119],
       ...,
       [0.12110367, 0.07446129, 0.04558758, ..., 1.        , 0.07085128,
        0.0196177 ],
       [0.07347567, 0.16773875, 0.04938579, ..., 0.07085128, 1.        ,
        0.10602962],
       [0.04316046, 0.14263397, 0.10796119, ..., 0.0196177 , 0.10602962,
        1.        ]])

In [99]:
# Similarity score of book on index 0
similarity_score[0]

array([1.        , 0.10255025, 0.01220856, 0.        , 0.05367224,
       0.02774901, 0.08216491, 0.13732869, 0.03261686, 0.03667591,
       0.02322418, 0.06766487, 0.02083978, 0.09673735, 0.13388865,
       0.08303112, 0.11153543, 0.05100411, 0.02517784, 0.11706383,
       0.        , 0.14333793, 0.07847534, 0.06150451, 0.08723968,
       0.        , 0.07009814, 0.13658681, 0.07600328, 0.12167134,
       0.00768046, 0.01473221, 0.        , 0.07965814, 0.04522617,
       0.01556271, 0.09495938, 0.0182307 , 0.02610465, 0.07984012,
       0.11679969, 0.0569124 , 0.08354155, 0.08471898, 0.08785938,
       0.05491435, 0.0548505 , 0.27026514, 0.09779123, 0.06016046,
       0.08958835, 0.06748675, 0.        , 0.04468098, 0.01920872,
       0.        , 0.05629067, 0.00557964, 0.07877059, 0.05219479,
       0.18908177, 0.        , 0.01240656, 0.02984572, 0.04279502,
       0.12680125, 0.16566735, 0.        , 0.13357242, 0.06615478,
       0.        , 0.        , 0.        , 0.10968075, 0.02806

In [158]:
def recommend_books(book_name):
    recommended_books = []
    
    index = list(pt.index).index(book_name)
    similar_books = sorted(list(enumerate(similarity_score[index])), key=lambda x:x[1], reverse=True)[1:10]
    
    for book in similar_books:
        recommended_books.append(pt.index[book[0]])
    
    return recommended_books

In [159]:
similar_books = recommend_books('1984')
similar_books

['Animal Farm',
 "The Handmaid's Tale",
 'Brave New World',
 'The Vampire Lestat (Vampire Chronicles, Book II)',
 'The Hours : A Novel',
 'Fahrenheit 451',
 'The Catcher in the Rye',
 'Naked',
 'The Hundred Secret Senses']

In [3]:
# For flask web app
def recommend_books(book_name):
    recommended_books = []
    
    index = list(pt.index).index(book_name)
    similar_books = sorted(list(enumerate(similarity_score[index])), key=lambda x:x[1], reverse=True)[1:10]
    
    for book in similar_books:
        item = {}
        temp_df = books[books['Book-Title'] == pt.index[book[0]]].drop_duplicates('Book-Title')
        item['Book_Name'] = temp_df['Book-Title'].iloc[0]
        item['Book_Author'] = temp_df['Book-Author'].iloc[0]
        item['Image_URL'] = temp_df['Image-URL-M'].iloc[0]
        item['Book_Ratings'] = avg_ratings[avg_ratings['Book-Title'] == pt.index[book[0]]]['Avg-Ratings'].iloc[0]
        
        recommended_books.append(item)
    
    return recommended_books

In [4]:
similar_books = recommend_books('1984')
similar_books

[{'Book_Name': 'Animal Farm',
  'Book_Author': 'George Orwell',
  'Image_URL': 'http://images.amazon.com/images/P/0451526341.01.MZZZZZZZ.jpg',
  'Book_Ratings': 4.274678111587983},
 {'Book_Name': "The Handmaid's Tale",
  'Book_Author': 'Margaret Atwood',
  'Image_URL': 'http://images.amazon.com/images/P/0449212602.01.MZZZZZZZ.jpg',
  'Book_Ratings': 3.3987138263665595},
 {'Book_Name': 'Brave New World',
  'Book_Author': 'Aldous Huxley',
  'Image_URL': 'http://images.amazon.com/images/P/0060809833.01.MZZZZZZZ.jpg',
  'Book_Ratings': 4.331858407079646},
 {'Book_Name': 'The Vampire Lestat (Vampire Chronicles, Book II)',
  'Book_Author': 'ANNE RICE',
  'Image_URL': 'http://images.amazon.com/images/P/0345313860.01.MZZZZZZZ.jpg',
  'Book_Ratings': 3.777408637873754},
 {'Book_Name': 'The Hours : A Novel',
  'Book_Author': 'Michael Cunningham',
  'Image_URL': 'http://images.amazon.com/images/P/0312243022.01.MZZZZZZZ.jpg',
  'Book_Ratings': 3.4444444444444446},
 {'Book_Name': 'Fahrenheit 451',


In [197]:
import pickle

with open('pt.pkl', 'wb') as f:
    pickle.dump(pt, f)
    
with open('books.pkl', 'wb') as f:
    pickle.dump(books, f)
    
with open('avg_ratings.pkl', 'wb') as f:
    pickle.dump(avg_ratings, f)
    
with open('similarity_score.pkl', 'wb') as f:
    pickle.dump(similarity_score, f)